# **Use case ML: price prediction of used bikes**

**Table of contents**<a id='toc0_'></a>    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## *Data Preprocessing*

In [19]:
########### Loading in Libs ###########

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, minmax_scale, MaxAbsScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, learning_curve, KFold
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, r2_score, make_scorer, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectFromModel
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import pickle


########### Style for Plots  ###########

plt.style.use('ggplot')

np.int = np.int_ #fix needed for the py version i used

In [20]:
########### reading in the dataframes ###########
df_BMW_raw = pd.read_csv("BMW_bike.csv", delimiter = ",", usecols=[0, 1, 2, 3], dtype={"mileage":"string", "price":"string", "Bike":"string", "Types and Used Time": "string"})
df_ducatti_raw = pd.read_csv("ducatti_bike.csv", delimiter = ",", usecols=[0, 1, 2, 3])
df_KTM_raw = pd.read_csv("KTM_bike.csv", delimiter = ",", usecols=[0, 1, 2, 3])
df_Royal_Enfield_raw = pd.read_csv("Royal_Enfield_Standard_bike.csv", delimiter = ",", usecols=[0, 1, 2, 3])
df_Suzuki_raw = pd.read_csv("Suzuki_bike.csv", delimiter = ",",usecols=[0, 1, 2, 3])
df_Yamaha_raw = pd.read_csv("Yamaha_bike.csv", delimiter = ",",usecols=[0, 1, 2, 3])
df_bikez =  pd.read_csv("all_bikez_curated.csv", delimiter = ",", dtype={"Brand":"string", "Model":"string", "Year":int, "Category":"string", "Rating":float, "Displacement (ccm)":float, "Power (hp)":float, "Torque(Nm)":float, "Engine cylinder":"string", "Engine stroke":"string",'Gearbox':"string", 'Bore (mm)':float, 'Stroke (mm)':"string", 'Fuel capacity (lts)':float,
       'Fuel system':"string", 'Fuel control':"string", 'Cooling system':"string", 'Transmission type':"string",
       'Dry weight (kg)':float, 'Wheelbase (mm)':float, 'Seat height (mm)':float, 'Front brakes':"string",
       'Rear brakes':"string", 'Front tire':"string", 'Rear tire':"string", 'Front suspension':"string",
       'Rear suspension':"string", 'Color options':"string"})

########### Dropping nans for mileage and formatting bike column to lower case ###########
df_BMW_raw = df_BMW_raw.dropna(subset=["mileage"])
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.lower()

df_ducatti_raw = df_ducatti_raw.dropna(subset=["mileage"])
df_ducatti_raw = df_ducatti_raw.rename(columns={"Bike name": "Bike", "Time of USed": "Types and Used Time"})
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.lower()

df_KTM_raw = df_KTM_raw.dropna(subset=["mileage"])
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.lower()

df_Royal_Enfield_raw = df_Royal_Enfield_raw.dropna(subset=["mileage"])
df_Royal_Enfield_raw = df_Royal_Enfield_raw.rename(columns={"bike": "Bike", "Types": "Types and Used Time"})
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.lower()

df_Suzuki_raw = df_Suzuki_raw.dropna(subset=["mileage"])
df_Suzuki_raw = df_Suzuki_raw.rename(columns={"BIke name": "Bike"})
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.lower()

df_Yamaha_raw = df_Yamaha_raw.dropna(subset=["mileage"])
df_Yamaha_raw = df_Yamaha_raw.rename(columns={"Bike name": "Bike", "Types and Used  Time": "Types and Used Time"})
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.lower()

########### Rename models column to Bike column and reformat ###########
df_bikez = df_bikez.rename(columns={"Model": "Bike"})
df_bikez["Bike"] = df_bikez["Bike"].str.lower()

In [21]:
########### Creating dataframes for the model specifications for the different brands for performance and overview reasons ###########

df_bikez_bmw = df_bikez[df_bikez["Brand"] == "bmw"]
df_bikez_royal_enfield = df_bikez[df_bikez["Brand"] == "enfield"]
df_bikez_ducatti = df_bikez[df_bikez["Brand"] == "ducati"]
df_bikez_ktm = df_bikez[df_bikez["Brand"] == "ktm"]
df_bikez_suzuki = df_bikez[df_bikez["Brand"] == "suzuki"]
df_bikez_yamaha = df_bikez[df_bikez["Brand"] == "yamaha"]

### BMW:

In [22]:
########### Extract Year ###########

df_BMW_raw["Year"] = df_BMW_raw["Types and Used Time"].str.split('BMW').str[0]
df_BMW_raw["Year"] = df_BMW_raw["Year"].replace("New", "", regex=True).astype(int)

########### Reformat ###########

df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace(" ", "", regex=True)
df_bikez_bmw["Bike"] = df_bikez_bmw["Bike"].replace(" ", "", regex=True)

########### Adjust model names to fit the names from the model specifications dataframe ###########

df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.split('first').str[0]
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace("[/]", "", regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace("r1100sboxercup", "r1100sboxercupreplica", regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace("r1250", "r1250gs", regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace("r1100rtabs", "r1100rt", regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].replace("r807", "r80", regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1250gs.*$)', 'r1250gs', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*s1000xr.*$)', 's1000xr', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*g310.*$)', 'g310r', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*k1600b.*$)', 'k1600b', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1250.*$)', 'r1250gs', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r18.*$)', 'r18', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*s1000r.*$)', 's1000r', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f900r.*$)', 'f900r', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f900xr.*$)', 'f900xr', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1200rt.*$)', 'r1200rt', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*ce04.*$)', 'ce04', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1200gsadventure.*$)', 'r1200gsadventure', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f850gsadventure.*$)', 'f850gsadventure', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f800gt.*$)', 'f800gt', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f800gs.*$)', 'f800gs', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*s1000.*$)', 's1000rr', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1250rt.*$)', 'r1250rt', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*m1000r.*$)', 'm1000rr', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f750.*$)', 'f750gs', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*f850gs.*$)', 'f850gs', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*k1600.*$)', 'k1600b', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1000.*$)', 'm1000rr', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1200.*$)', 'r1200avantgarde', regex=True)
df_BMW_raw["Bike"] = df_BMW_raw["Bike"].str.replace(r'(^.*r1200gs.*$)', 'r1200gs', regex=True)

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1450460988.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_bmw["Bike"] = df_bikez_bmw["Bike"].replace(" ", "", regex=True)


In [23]:
########### Merge according to Bike and Year ###########

df_BMW = pd.merge(df_BMW_raw, df_bikez_bmw, how = "left",  on = ["Bike", "Year"])

########### Drop the with that created duplicates ###########

df_BMW = df_BMW.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

########### Merge on bike model for the cases for which there is no corresponding Year ###########

df_merged_temp = pd.merge(df_BMW_raw, df_bikez_bmw, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_BMW.reset_index(inplace=True, drop=True)

########### Fill NaNs for the same model ###########

df_BMW["Category"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_BMW["Displacement (ccm)"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_BMW["Power (hp)"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_BMW["Torque (Nm)"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_BMW["Engine stroke"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
df_BMW["Brand"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

########### Drop entries for which the Category is empty ###########

df_BMW = df_BMW.dropna(subset=["Category"])

########### Drop unused columns ###########

df_BMW = df_BMW.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

########### Drop duplicates again? This kinda grew historically throughout the programming. It had a reason at some point I think :) ###########

df_BMW = df_BMW.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

########### Fill NaNs for which some entries are empty and there is no corresponding model with values ###########

df_BMW["Displacement (ccm)"].fillna(value=df_BMW.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_BMW["Displacement (ccm)"].fillna(value=df_BMW["Displacement (ccm)"].mean(), inplace=True) 

df_BMW["Power (hp)"].fillna(value=df_BMW.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_BMW["Power (hp)"].fillna(value=df_BMW["Power (hp)"].mean(), inplace=True) 

df_BMW["Torque (Nm)"].fillna(value=df_BMW.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_BMW["Torque (Nm)"].fillna(value=df_BMW["Torque (Nm)"].mean(), inplace=True) 

########### Reformat mileage and price + drop entries for which there is no price ###########

df_BMW["mileage"] = df_BMW["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_BMW = df_BMW.loc[df_BMW["price"]!= "No Price Listed"]
df_BMW["price"] = df_BMW["price"].replace("[$,]", "", regex=True).astype(int)


/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/3030007067.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_BMW["Category"].fillna(df_BMW['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/3030007067.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work becaus

$\textbf{BMW CLEANING DONE}$

### Royal Enfield:

In [24]:
df_Royal_Enfield_raw["Year"] = df_Royal_Enfield_raw["Types and Used Time"].str.split('Royal').str[0]
df_Royal_Enfield_raw["Year"] = df_Royal_Enfield_raw["Year"].replace("New", "", regex=True)
df_Royal_Enfield_raw["Year"] = df_Royal_Enfield_raw["Year"].replace("Used", "", regex=True).astype(int)

df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].replace(" ", "", regex=True)
df_bikez_royal_enfield["Bike"] = df_bikez_royal_enfield["Bike"].replace(" ", "", regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*himalayan.*$)', 'himalayan', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*continentalgt.*$)', 'continentalgt', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*classic350.*$)', 'classic350', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*int650.*$)', 'interceptor650', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*meteor350.*$)', 'meteor350', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*classic500.*$)', 'classic500', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*metro350.*$)', 'meteor350', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*contienetalgt650.*$)', 'continentalgt650', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*meteor.*$)', 'meteor350', regex=True)
df_Royal_Enfield_raw["Bike"] = df_Royal_Enfield_raw["Bike"].str.replace(r'(^.*interceptor650.*$)', 'interceptor650', regex=True)

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/746029178.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_royal_enfield["Bike"] = df_bikez_royal_enfield["Bike"].replace(" ", "", regex=True)


In [25]:
df_Royal_Enfield = pd.merge(df_Royal_Enfield_raw, df_bikez_royal_enfield, how = "left",  on = ["Bike", "Year"])
df_Royal_Enfield = df_Royal_Enfield.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

df_merged_temp = pd.merge(df_Royal_Enfield_raw, df_bikez_royal_enfield, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_Royal_Enfield.reset_index(inplace=True, drop=True)

df_Royal_Enfield["Category"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_Royal_Enfield["Displacement (ccm)"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_Royal_Enfield["Power (hp)"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_Royal_Enfield["Torque (Nm)"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_Royal_Enfield["Engine stroke"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
#df_Royal_Enfield["Dry weight (kg)"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Dry weight (kg)']), inplace=True) 
df_Royal_Enfield["Brand"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

df_Royal_Enfield = df_Royal_Enfield.dropna(subset=["Category"])

df_Royal_Enfield = df_Royal_Enfield.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

#df_Royal_Enfield["Dry weight (kg)"].fillna(value=df_BMW.groupby(['Category'])['Dry weight (kg)'].mean(), inplace=True)
#df_Royal_Enfield["Dry weight (kg)"].fillna(value=df_BMW['Dry weight (kg)'].mean(), inplace=True)

df_Royal_Enfield["Displacement (ccm)"].fillna(value=df_Royal_Enfield.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_Royal_Enfield["Displacement (ccm)"].fillna(value=df_Royal_Enfield["Displacement (ccm)"].mean(), inplace=True) 

df_Royal_Enfield["Power (hp)"].fillna(value=df_Royal_Enfield.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_Royal_Enfield["Power (hp)"].fillna(value=df_Royal_Enfield["Power (hp)"].mean(), inplace=True) 

df_Royal_Enfield["Torque (Nm)"].fillna(value=df_Royal_Enfield.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_Royal_Enfield["Torque (Nm)"].fillna(value=df_Royal_Enfield["Torque (Nm)"].mean(), inplace=True) 

df_Royal_Enfield["mileage"] = df_Royal_Enfield["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_Royal_Enfield = df_Royal_Enfield.loc[df_Royal_Enfield["price"]!= "No Price Listed"]
df_Royal_Enfield["price"] = df_Royal_Enfield["price"].replace("[$,]", "", regex=True).astype(int)
df_Royal_Enfield

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1737067518.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Royal_Enfield["Category"].fillna(df_Royal_Enfield['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1737067518.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

,mileage,price,Bike,Types and Used Time,Year,Brand,Category,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke
0,1,5299,himalayan,New 2022 Royal Enfield Standard,2022,enfield,Enduro / offroad,411.0,24.500000,32.0,Single cylinder,four-stroke
1,1256,4500,bullet500b5,Used 2009 Royal Enfield Classic / Vintage,2009,enfield,Classic,499.0,30.218505,40.9,<NA>,four-stroke
2,5,6349,continentalgt,New 2023 Royal Enfield Cruiser,2023,enfield,Allround,535.0,29.100000,41.3,<NA>,four-stroke
3,3,6349,continentalgt,New 2023 Royal Enfield Cruiser,2023,enfield,Allround,535.0,29.100000,41.3,<NA>,four-stroke
4,1,4499,classic350,New 2022 Royal Enfield Standard,2022,enfield,Classic,349.3,19.700000,27.0,Single cylinder,four-stroke
...,...,...,...,...,...,...,...,...,...,...,...,...
338,5717,4700,meteor350,Used 2021 Royal Enfield Cruiser,2021,enfield,Classic,349.0,19.900000,27.0,<NA>,four-stroke
339,1056,3980,meteor350,Used 2021 Royal Enfield Cruiser,2021,enfield,Classic,349.0,19.900000,27.0,<NA>,four-stroke
340,1162,4495,interceptor650,Used 2019 Royal Enfield Standard,2019,enfield,Allround,648.0,47.000000,52.0,Twin,four-stroke
341,6,5449,himalayan,New 2023 Royal Enfield Standard,2023,enfield,Enduro / offroad,411.0,24.500000,32.0,<NA>,four-stroke


$\textbf{ROYAL ENFIELD CLEANING DONE}$

### Ducati:

In [26]:
df_ducatti_raw["Year"] = df_ducatti_raw["Types and Used Time"].str.split('Ducati').str[0]
df_ducatti_raw["Year"] = df_ducatti_raw["Year"].replace("New", "", regex=True)
df_ducatti_raw["Year"] = df_ducatti_raw["Year"].replace("Used", "", regex=True).astype(int)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].replace(" ", "", regex=True)
df_bikez_ducatti["Bike"] = df_bikez_ducatti["Bike"].replace(" ", "", regex=True)

df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*superleggera1299.*$)', 'superleggera1299', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*1199panigale.*$)', '1199panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*937.*$)', 'monster797plus', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigalev4sp.*$)', 'panigalev4sp', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*monster821.*$)', 'monster821', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*959panigale.*$)', '959panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].replace("superbikepanigalev4", "panigalev4", regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*multistradav4.*$)', 'multistradav4', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*diavel.*$)', 'diavel1260', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*multistrada1260.*$)', 'multistrada1260', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*base.*$)', 'monsterplus', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigale899.*$)', 'panigale959', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigalev2.*$)', 'panigalev2', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*superleggera.*$)', 'superleggerav4', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*monster1200.*$)', 'monster1200', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*scrambler800.*$)', 'scramblercaferacer', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*streetfighterv4s.*$)', 'streetfighterv4s', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*1299panigale.*$)', '1299panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*supersport9.*$)', 'supersport950', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*monster+.*$)', 'monsterplus', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigale1199.*$)', '1199panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigale1299.*$)', '1199panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*multistrada1200.*$)', 'multistrada1200enduro', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*multistrada950.*$)', 'multistrada950', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*899panigale.*$)', '899panigale', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*streetfighterv4d.*$)', 'streetfighterv4', regex=True)
df_ducatti_raw["Bike"] = df_ducatti_raw["Bike"].str.replace(r'(^.*panigalev4s.*$)', 'panigalev4s', regex=True)


/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1497480933.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_ducatti["Bike"] = df_bikez_ducatti["Bike"].replace(" ", "", regex=True)


In [27]:
df_ducati = pd.merge(df_ducatti_raw, df_bikez_ducatti, how = "left",  on = ["Bike", "Year"])
df_ducati = df_ducati.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

df_merged_temp = pd.merge(df_ducatti_raw, df_bikez_ducatti, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_ducati.reset_index(inplace=True, drop=True)

df_ducati["Category"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_ducati["Displacement (ccm)"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_ducati["Power (hp)"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_ducati["Torque (Nm)"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_ducati["Engine stroke"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
#df_ducati["Dry weight (kg)"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Dry weight (kg)']), inplace=True) 
df_ducati["Brand"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

df_ducati = df_ducati.dropna(subset=["Category"])

df_ducati = df_ducati.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

#df_ducati["Dry weight (kg)"].fillna(value=df_ducati.groupby(['Category'])['Dry weight (kg)'].mean(), inplace=True)
#df_ducati["Dry weight (kg)"].fillna(value=df_ducati['Dry weight (kg)'].mean(), inplace=True)

df_ducati["Displacement (ccm)"].fillna(value=df_ducati.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_ducati["Displacement (ccm)"].fillna(value=df_ducati["Displacement (ccm)"].mean(), inplace=True) 

df_ducati["Power (hp)"].fillna(value=df_ducati.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_ducati["Power (hp)"].fillna(value=df_ducati["Power (hp)"].mean(), inplace=True) 

df_ducati["Torque (Nm)"].fillna(value=df_ducati.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_ducati["Torque (Nm)"].fillna(value=df_ducati["Torque (Nm)"].mean(), inplace=True) 

df_ducati["mileage"] = df_ducati["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_ducati = df_ducati.loc[df_ducati["price"]!= "No Price Listed"]
df_ducati["price"] = df_ducati["price"].replace("[$,]", "", regex=True).astype(int)
df_ducati

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1127063342.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ducati["Category"].fillna(df_ducati['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1127063342.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work b

,mileage,price,Bike,Types and Used Time,Year,Brand,Category,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke
0,1050,49995,superleggerav4,2014 Ducati Sportbike,2014,ducati,Naked bike,998.0,221.0,116.0,<NA>,four-stroke
1,6308,11795,monsterplus,2021 Ducati Dirt Bike,2021,ducati,Naked bike,937.0,111.0,93.0,V2,four-stroke
2,2400,9495,scrambler1100,2018 Ducati Sportbike,2018,ducati,Classic,1079.0,84.5,88.0,V2,four-stroke
3,11200,5500,st2,1999 Ducati Sport Touring,1999,ducati,Sport touring,944.0,83.0,83.0,V2,four-stroke
4,1300,34900,streetfighterv4s,2022 Ducati Sportbike,2022,ducati,Naked bike,1103.0,208.0,123.0,<NA>,four-stroke
...,...,...,...,...,...,...,...,...,...,...,...,...
578,2049,10499,scramblercaferacer,2018 Ducati Standard,2018,ducati,Classic,803.0,73.0,68.0,V2,four-stroke
579,20632,14495,multistrada1200enduro,2016 Ducati Sport Touring,2016,ducati,Enduro / offroad,1198.4,160.0,136.0,V2,four-stroke
580,10850,11999,diavel1260,2015 Ducati Sportbike,2015,ducati,Allround,1262.0,155.0,129.0,<NA>,four-stroke
581,12000,10750,hypermotardsp,2015 Ducati Super Moto,2015,ducati,Super motard,821.1,110.0,89.0,V2,four-stroke


$\textbf{DUCATTI CLEANING DONE}$

### KTM:

In [28]:
df_KTM_raw["Year"] = df_KTM_raw["Types and Used Time"].str.split('KTM').str[0]
df_KTM_raw["Year"] = df_KTM_raw["Year"].replace("New", "", regex=True)
df_KTM_raw["Year"] = df_KTM_raw["Year"].replace("Used", "", regex=True).astype(int)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].replace(" ", "", regex=True)
df_bikez_ktm["Bike"] = df_bikez_ktm["Bike"].replace(" ", "", regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*smc690r.*$)', '690smcr', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*690smcr.*$)', '690smcr', regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*superduke1290r.*$)', '1290superduker', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*1290superduke.*$)', '1290superduker', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*superduke990.*$)', '990superduke', regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*duke390abs.*$)', '390duke', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*adventure1190.*$)', '1090adventure', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*adventure1290.*$)', '1290superadventurer', regex=True)
#df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*adventure1290.*$)', '1290adventure', regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*adventure890.*$)', '890adventure', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*890adventure.*$)', '890adventure', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*790adventure.*$)', '790adventure', regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*890duke.*$)', '890duke', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*rc390.*$)', 'rc390', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*690duke.*$)', '690duke', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*duke690.*$)', '690duke', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*500exc-f.*$)', '500exc-f', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*450xc.*$)', '450exc-f', regex=True)
df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*350xc.*$)', '350exc-f', regex=True)

df_KTM_raw["Bike"] = df_KTM_raw["Bike"].str.replace(r'(^.*smc690r.*$)', '690smcr', regex=True)

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1294204431.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_ktm["Bike"] = df_bikez_ktm["Bike"].replace(" ", "", regex=True)


In [29]:
df_ktm = pd.merge(df_KTM_raw, df_bikez_ktm, how = "left",  on = ["Bike", "Year"])
df_ktm = df_ktm.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

df_merged_temp = pd.merge(df_KTM_raw, df_bikez_ktm, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_ktm.reset_index(inplace=True, drop=True)

df_ktm["Category"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_ktm["Displacement (ccm)"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_ktm["Power (hp)"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_ktm["Torque (Nm)"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_ktm["Engine stroke"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
#df_ktm["Dry weight (kg)"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Dry weight (kg)']), inplace=True) 
df_ktm["Brand"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

df_ktm = df_ktm.dropna(subset=["Category"])

df_ktm = df_ktm.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

#df_ktm["Dry weight (kg)"].fillna(value=df_ktm.groupby(['Category'])['Dry weight (kg)'].mean(), inplace=True)
#df_ktm["Dry weight (kg)"].fillna(value=df_ktm['Dry weight (kg)'].mean(), inplace=True)

df_ktm["Displacement (ccm)"].fillna(value=df_ktm.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_ktm["Displacement (ccm)"].fillna(value=df_ktm["Displacement (ccm)"].mean(), inplace=True) 

df_ktm["Power (hp)"].fillna(value=df_ktm.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_ktm["Power (hp)"].fillna(value=df_ktm["Power (hp)"].mean(), inplace=True) 

df_ktm["Torque (Nm)"].fillna(value=df_ktm.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_ktm["Torque (Nm)"].fillna(value=df_ktm["Torque (Nm)"].mean(), inplace=True) 

df_ktm["mileage"] = df_ktm["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_ktm = df_ktm.loc[df_ktm["price"]!= "No Price Listed"]
df_ktm["price"] = df_ktm["price"].replace("[$,]", "", regex=True).astype(int)
df_ktm

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1960192824.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ktm["Category"].fillna(df_ktm['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/1960192824.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because

,mileage,price,Bike,Types and Used Time,Year,Brand,Category,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke
0,10336,14994,1290superduker,2019 KTM Sport Touring,2019,ktm,Naked bike,1301.0,177.000000,87.525114,V2,four-stroke
1,5796,10999,690smcr,2019 KTM Super Moto,2019,ktm,Super motard,690.0,67.000000,64.000000,Single cylinder,four-stroke
2,485,9500,350exc-f,2018 KTM Dual Sport,2018,ktm,Enduro / offroad,349.7,109.491579,87.525114,Single cylinder,four-stroke
3,57,7995,350exc-f,2020 KTM Dual Sport,2020,ktm,Enduro / offroad,349.7,109.491579,87.525114,Single cylinder,four-stroke
5,35,9586,790duke,2019 KTM Standard,2019,ktm,Naked bike,799.0,105.000000,87.525114,Twin,four-stroke
...,...,...,...,...,...,...,...,...,...,...,...,...
583,26756,9999,1190adventurer,2014 KTM Dual Sport,2014,ktm,Allround,1195.0,147.000000,87.525114,Twin,four-stroke
584,11815,5999,690absduke,2014 KTM Sportbike,2014,ktm,Naked bike,690.0,67.100000,87.525114,Single cylinder,four-stroke
585,13568,10999,1190adventure,2014 KTM Dual Sport,2014,ktm,Allround,1195.0,147.000000,87.525114,Twin,four-stroke
586,10995,4499,690duke,2014 KTM Sportbike,2014,ktm,Naked bike,654.0,64.400000,67.000000,<NA>,four-stroke


$\textbf{KTM CLEANING DONE}$

### Suzuki:

In [30]:
df_Suzuki_raw["Year"] = df_Suzuki_raw["Types and Used Time"].str.split('Suzuki').str[0]
df_Suzuki_raw["Year"] = df_Suzuki_raw["Year"].replace("New", "", regex=True)
df_Suzuki_raw["Year"] = df_Suzuki_raw["Year"].replace("Used", "", regex=True).astype(int)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].replace(" ", "", regex=True)
df_bikez_suzuki["Bike"] = df_bikez_suzuki["Bike"].replace(" ", "", regex=True)

df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*hayabusa.*$)', 'hayabusa', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-r1300.*$)', 'hayabusa', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-1300r.*$)', 'hayabusa', regex=True)

df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx1300.*$)', 'hayabusa', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-1000.*$)', 'gsx-r1000', regex=True)

df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*m109rb.*$)', 'boulevardm109rboss', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*boulevard.*$)', 'boulevardc50t', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxr1000.*$)', 'gsx-r1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-r1000.*$)', 'gsx-r1000', regex=True)

df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-r750.*$)', 'gsx-r750', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxr750.*$)', 'gsx-r750', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxr600.*$)', 'gsx-r600', regex=True)

df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*vstrom650.*$)', 'v-strom650', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*vstrom1000.*$)', 'v-strom1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*v-strom1000.*$)', 'v-strom1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*v-strom1000.*$)', 'v-strom1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*bandit1200.*$)', 'bandit1250sa', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-s1000gt.*$)', 'gsx-s1000gt', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx1000.*$)', 'gsx-r1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsx-s750z.*$)', 'gsx-s750z', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxs1000.*$)', 'gsx-s1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxs1000.*$)', 'gsx-s1000', regex=True)
df_Suzuki_raw["Bike"] = df_Suzuki_raw["Bike"].str.replace(r'(^.*gsxr-600.*$)', 'gsx-r600', regex=True)



/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/2620846397.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_suzuki["Bike"] = df_bikez_suzuki["Bike"].replace(" ", "", regex=True)


In [31]:
df_suzuki = pd.merge(df_Suzuki_raw, df_bikez_suzuki, how = "left",  on = ["Bike", "Year"])
df_suzuki = df_suzuki.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

df_merged_temp = pd.merge(df_Suzuki_raw, df_bikez_suzuki, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_suzuki.reset_index(inplace=True, drop=True)

df_suzuki["Category"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_suzuki["Displacement (ccm)"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_suzuki["Power (hp)"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_suzuki["Torque (Nm)"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_suzuki["Engine stroke"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
#df_suzuki["Dry weight (kg)"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Dry weight (kg)']), inplace=True) 
df_suzuki["Brand"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

df_suzuki = df_suzuki.dropna(subset=["Category"])

df_suzuki = df_suzuki.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

#df_suzuki["Dry weight (kg)"].fillna(value=df_ktm.groupby(['Category'])['Dry weight (kg)'].mean(), inplace=True)
#df_suzuki["Dry weight (kg)"].fillna(value=df_ktm['Dry weight (kg)'].mean(), inplace=True)

df_suzuki["Displacement (ccm)"].fillna(value=df_suzuki.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_suzuki["Displacement (ccm)"].fillna(value=df_suzuki["Displacement (ccm)"].mean(), inplace=True) 

df_suzuki["Power (hp)"].fillna(value=df_suzuki.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_suzuki["Power (hp)"].fillna(value=df_suzuki["Power (hp)"].mean(), inplace=True) 

df_suzuki["Torque (Nm)"].fillna(value=df_suzuki.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_suzuki["Torque (Nm)"].fillna(value=df_suzuki["Torque (Nm)"].mean(), inplace=True) 

df_suzuki["mileage"] = df_suzuki["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_suzuki = df_suzuki.loc[df_suzuki["price"]!= "No Price Listed"]
df_suzuki["price"] = df_suzuki["price"].replace("[$,]", "", regex=True).astype(int)
df_suzuki

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/876307532.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_suzuki["Category"].fillna(df_suzuki['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/876307532.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work bec

,mileage,price,Bike,Types and Used Time,Year,Brand,Category,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke
0,8038,14995,gsx-r1000,2020 Suzuki Sportbike,2020,suzuki,Sport,999.8,198.000000,117.000000,In-line four,four-stroke
1,10217,10498,boulevardc50t,2019 Suzuki Cruiser,2019,suzuki,Touring,805.0,53.000000,69.000000,V2,four-stroke
2,18,18500,hayabusa,2022 Suzuki Sportbike,2022,suzuki,Sport,1340.0,197.000000,155.000000,In-line four,four-stroke
3,10600,6450,v-strom650absadventure,2013 Suzuki Dual Sport,2013,suzuki,Super motard,645.0,67.000000,60.000000,V2,four-stroke
4,5540,6999,dr-z400sm,2021 Suzuki Competition,2021,suzuki,Super motard,398.0,110.543925,89.722047,Single cylinder,four-stroke
...,...,...,...,...,...,...,...,...,...,...,...,...
748,3,10799,v-strom650,New 2023 Suzuki Dual Sport,2023,suzuki,Allround,645.0,110.543925,89.722047,<NA>,four-stroke
749,5,15599,boulevardc50t,New 2023 Suzuki Cruiser,2023,suzuki,Custom / cruiser,819.4,110.543925,89.722047,<NA>,four-stroke
750,2,9104,v-strom650,New 2023 Suzuki Dual Sport,2023,suzuki,Allround,645.0,110.543925,89.722047,<NA>,four-stroke
751,2,18799,hayabusa,New 2023 Suzuki Sportbike,2023,suzuki,Sport,1340.0,110.543925,89.722047,<NA>,four-stroke


$\textbf{SUZUKI CLEANING DONE}$

### Yamaha:

In [32]:
df_Yamaha_raw["Year"] = df_Yamaha_raw["Types and Used Time"].str.split('Yamaha').str[0]
df_Yamaha_raw["Year"] = df_Yamaha_raw["Year"].replace("New", "", regex=True)
df_Yamaha_raw["Year"] = df_Yamaha_raw["Year"].replace("Used", "", regex=True).astype(int)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].replace(" ", "", regex=True)
df_bikez_yamaha["Bike"] = df_bikez_yamaha["Bike"].replace(" ", "", regex=True)

df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr1m.*$)', 'yzf-r1m', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt-03.*$)', 'mt-03', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr6.*$)', 'yzf-r6', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xvs1100.*$)', 'xvs950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xvs650.*$)', 'xvs650', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xvs950.*$)', 'xvs950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xv1900.*$)', 'xv1900a', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xv1700.*$)', 'xv1900a', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*fz07.*$)', 'fz-07', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*raider1900.*$)', 'raider', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yz450f.*$)', 'yz450f', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*vstar950.*$)', 'vstar950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*fz-07.*$)', 'fz-07', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt03.*$)', 'mt-03', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xv650.*$)', 'xvs650', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*xvs1300.*$)', 'xvs950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzf-r7.*$)', 'yzf-r7', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr3.*$)', 'yzf-r3', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt09.*$)', 'mt-09', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*r1yzf.*$)', 'yzf-r1m', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*raider.*$)', 'raider', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*v-star1100.*$)', 'vstar950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt09.*$)', 'mt-09', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt10.*$)', 'mt-10', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr7.*$)', 'yzf-r7', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzf-r3.*$)', 'yzf-r3', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr7.*$)', 'yzf-r7', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr150.*$)', 'yzf-r15', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*vstar650.*$)', 'vstar650', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*vstarcustom650.*$)', 'vstar650', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt-10ca.*$)', 'mt-10', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt-07ca.*$)', 'mt-07', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*vstar®1100.*$)', 'vstar950', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*yzfr1s.*$)', 'yzf-r1s', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*fz10.*$)', 'fz-10', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*fz09.*$)', 'fz-09', regex=True)
df_Yamaha_raw["Bike"] = df_Yamaha_raw["Bike"].str.replace(r'(^.*mt07.*$)', 'mt-07', regex=True)



/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/4220114532.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bikez_yamaha["Bike"] = df_bikez_yamaha["Bike"].replace(" ", "", regex=True)


In [33]:
df_yamaha = pd.merge(df_Yamaha_raw, df_bikez_yamaha, how = "left",  on = ["Bike", "Year"])
df_yamaha = df_yamaha.drop_duplicates(subset = ["mileage", "price", "Bike", "Year"], keep = "first")

df_merged_temp = pd.merge(df_Yamaha_raw, df_bikez_yamaha, how = "left",  on = ["Bike"])
df_merged_temp = df_merged_temp.drop_duplicates(subset = ["Bike"], keep = "first") #jedes Modell ein uniquer Index
df_merged_temp.reset_index(inplace=True, drop=True)
df_yamaha.reset_index(inplace=True, drop=True)

df_yamaha["Category"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
df_yamaha["Displacement (ccm)"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Displacement (ccm)']), inplace=True) 
df_yamaha["Power (hp)"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Power (hp)']), inplace=True) 
df_yamaha["Torque (Nm)"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Torque (Nm)']), inplace=True) 
df_yamaha["Engine stroke"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Engine stroke']), inplace=True) 
#df_yamaha["Dry weight (kg)"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Dry weight (kg)']), inplace=True) 
df_yamaha["Brand"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Brand']), inplace=True) 

df_yamaha = df_yamaha.dropna(subset=["Category"])

df_yamaha = df_yamaha.drop(columns = ['Rating', 'Wheelbase (mm)', 'Seat height (mm)', 'Front brakes', 'Gearbox', 'Bore (mm)', 
       'Stroke (mm)','Fuel capacity (lts)', 'Fuel system','Fuel control', 'Cooling system', 'Transmission type' ,'Rear brakes', 'Front tire', 'Rear tire', 'Front suspension',
       'Rear suspension', 'Color options', 'Dry weight (kg)'])

#df_yamaha["Dry weight (kg)"].fillna(value=df_yamaha.groupby(['Category'])['Dry weight (kg)'].mean(), inplace=True)
#df_yamaha["Dry weight (kg)"].fillna(value=df_yamaha['Dry weight (kg)'].mean(), inplace=True)

df_yamaha["Displacement (ccm)"].fillna(value=df_yamaha.groupby('Category')["Displacement (ccm)"].mean(), inplace=True) 
df_yamaha["Displacement (ccm)"].fillna(value=df_yamaha["Displacement (ccm)"].mean(), inplace=True) 

df_yamaha["Power (hp)"].fillna(value=df_yamaha.groupby('Category')["Power (hp)"].mean(), inplace=True) 
df_yamaha["Power (hp)"].fillna(value=df_yamaha["Power (hp)"].mean(), inplace=True) 

df_yamaha["Torque (Nm)"].fillna(value=df_yamaha.groupby('Category')["Torque (Nm)"].mean(), inplace=True) 
df_yamaha["Torque (Nm)"].fillna(value=df_yamaha["Torque (Nm)"].mean(), inplace=True) 

df_yamaha["mileage"] = df_yamaha["mileage"].replace("[miles,]", "", regex=True).astype(int)
df_yamaha = df_yamaha.loc[df_yamaha["price"]!= "No Price Listed"]
df_yamaha["price"] = df_yamaha["price"].replace("[$,]", "", regex=True).astype(int)
df_yamaha

/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/3300373351.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_yamaha["Category"].fillna(df_yamaha['Bike'].map(df_merged_temp.set_index(['Bike'])['Category']), inplace=True)
/var/folders/zd/qsfm1f5d39ddz1bnq9k3xjwr0000gn/T/ipykernel_38067/3300373351.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work b

,mileage,price,Bike,Types and Used Time,Year,Brand,Category,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke
0,1261,16688,stareluder,2018 Yamaha Touring,2018,yamaha,Touring,1851.9,89.900000,171.000000,V2,four-stroke
1,11256,17999,starventure,2018 Yamaha Touring,2018,yamaha,Touring,1854.0,103.949842,81.543772,V2,four-stroke
3,9400,6500,fz-09,2015 Yamaha Sportbike,2015,yamaha,Naked bike,847.0,115.000000,61.500000,In-line three,four-stroke
4,6689,13499,supertenerees,2021 Yamaha Dual Sport,2021,yamaha,Super motard,1199.0,112.000000,117.000000,Twin,four-stroke
5,8415,6150,fz-07,2017 Yamaha Sportbike,2017,yamaha,Naked bike,689.0,103.949842,81.543772,Twin,four-stroke
...,...,...,...,...,...,...,...,...,...,...,...,...
643,1,8599,yz250f,New 2023 Yamaha Competition,2023,yamaha,Cross / motocross,249.0,103.949842,81.543772,<NA>,four-stroke
644,344,11500,yzf-r7,2023 Yamaha Sportbike,2023,yamaha,Sport,749.0,140.000000,81.543772,<NA>,four-stroke
645,1,1699,pw50,New 2023 Yamaha Dirt Bike,2023,yamaha,"Minibike, cross",49.0,2.700000,3.800000,<NA>,two-stroke
646,2,2299,tt-r110e,New 2023 Yamaha Dirt Bike,2023,yamaha,Enduro / offroad,110.0,103.949842,81.543772,<NA>,four-stroke


**SUZUKI CLEANING DONE**

### **MERGING DATAFRAMES:**

In [43]:
########### Merge dataframes ###########

df_all_brands = pd.concat([df_BMW, df_Royal_Enfield, df_ktm, df_ducati, df_suzuki, df_yamaha])

########### Drop further unused columns ###########

df_all_brands = df_all_brands.drop(columns = ["Types and Used Time", "Engine cylinder", "Engine stroke"])

########### Define Condition and Age + Drop Year ###########

df_all_brands["Condition"] = df_all_brands["mileage"] > 100 # True ist gebraucht, False ist neu
df_all_brands["Age"] = 2023 - df_all_brands["Year"]
df_all_brands = df_all_brands.drop(columns = ["Year"])

df_all_brands = df_all_brands.apply(lambda x: x.str.lower() if ((x.dtype == "object") | (x.dtype ==  "string[python]"))  else x)


df_all_brands.to_pickle("df_all_brands.pkl")